In [1]:
# This adds the path to import the development version (git repo) of NDI Python
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from ndi import Experiment, DaqSystem, FileNavigator
from ndi.daqreaders import MockReader
from ndi.database import FileSystem as Database
from ndi import Query as Q
from ndi.database.utils import print_everything_in, destroy_everything_in

In [3]:
# Connecting to a FileSystem database passing in a path to a directory to be used as the database
db = Database('../test_exp')
destroy_everything_in(db)

In [4]:
# Creating a FileNavigator object passing in regex patterns
fn = FileNavigator(epoch_file_patterns=['.*\\.wav', '.*\\.txt'], 
                   metadata_file_pattern='.*\\.txt')

In [5]:
# Mock EpochProbeMap class used just for this demo
class MockEpochProbeMap: pass

In [6]:
# Creating DaqSystem objects
ds = DaqSystem(name='myDaq1',
               file_navigator=fn,
               daq_reader=MockReader,
               epoch_probe_map=MockEpochProbeMap)
ds2 = DaqSystem(name='myDaq2',
                file_navigator=fn,
                daq_reader=MockReader,
                epoch_probe_map=MockEpochProbeMap)

In [7]:
# Instantiating Experiment objects and connecting it to its context (Database and DaqSystems)
e = Experiment(name='myExperiment3').connect(
    database=db,
    daq_systems=[ds, ds2]
)

In [8]:
# Testing that experiment (created above) was added to the database upon connection by 
# finding it based on its id
e2doc = db.find_by_id(e.id)

# Recreating the Experiment object from the found experiment document
e2 = Experiment.from_document(e2doc)

In [9]:
# Verifying the Experiments are the same
e2 == e

True

In [10]:
# Using the find method by passing in a query to get the original experiment 
# and verifying that its the same
e2 == Experiment.from_document(
    db.find(Q('_metadata.name') == 'myExperiment3')[0]
)

True

In [11]:
# Finding a non-match to show an empty list is returned
db.find(Q('_metadata.name') == 'myExperiment2')

[]

In [12]:
# Showing find method again with a query for DaqSystems
is_daq_system = Q('_metadata.type') == DaqSystem.DOCUMENT_TYPE
ds_results = db.find(is_daq_system)

In [13]:
print(ds_results)

[<ndi.document.Document object at 0x131372100>, <ndi.document.Document object at 0x131372310>]


In [14]:
print(ds_results[0].data)

# Verifying the DaqSystem found in the database matches this experiment's DaqSystem
e.ctx.daq_systems[0] == DaqSystem.from_document(ds_results[0])

{'_metadata': {'name': 'myDaq2', 'type': 'ndi_daq_system', 'experiment_id': '181219cf1ed84cb9a4d4b8bdd5adabd3', 'parent_id': '', 'asc_path': '', 'version_depth': 0, 'latest_version': True}, '_dependencies': {}, '_depends_on': [], 'epoch_ids': [], 'epoch_probe_map_class': 'MockEpochProbeMap', 'daq_reader_class': 'MockReader'}


False

In [15]:
# Passing nothing into find method to show that it returns all documents in the database
db.find()

In [16]:
# Deleting the created experiment by id
db.delete_by_id(e.id, force=True)

In [17]:
# Checking to make sure that the file is gone
result = db.find_by_id(e.id)
print(result is None)

True


In [18]:
# Deleting all items in the database because no query object was passed in
db.delete_many(force=True)

In [19]:
# Verifying that find method now returns an empty array, meaning all items have been deleted
db.find()

[]

In [20]:
# Adding a DaqSystem's document to the database
db.add(ds.document)

In [21]:
# Adding a DaqSystem's document to the database showing that an already existing document 
# can be upserted
db.upsert(ds.document, force=True)
db.find()

In [22]:
# Updating a document by id using update_by_id method, passing in a document's id and 
# a dictionary containing property changes
db.update_by_id(ds.id, { 'name': 'daq2' }, force=True)
db.find()

name: daq2


In [23]:
# Searching for the DaqSystem in the database
ds2 = db.find_by_id(ds.id)
db.find()

In [24]:
# Showing that the names match
print(ds.metadata['name'] == ds2.metadata['name'])

True


In [25]:
# Testing that wrong data type will result in error being raised
try:
    db.add('a')
except TypeError as e:
    print(e)

Unexpected type <class 'str'>. Expected type <class 'ndi.document.Document'>.
